In [8]:
import pandas as pd
import string
import numpy as np

data = pd.read_excel(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Punto de Venta'] + '-' + data['Número Desde']

# PROVEEDOR
data['Denominación Emisor'] = data['Denominación Emisor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Proveedor'] = data['Denominación Emisor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Emisor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']


# PONER 0 A TODAS LAS COLUMNAS
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])


# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)

# PONER EN NEGATICO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    data.loc[data['Tipo2'] == 'Crédito', col] = -data[col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','Factura',
        'Proveedor','CUIT',
        'NETO 10.5', 'IVA 10.5',
        'NETO 21', 'IVA 21', 'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,Factura,Proveedor,CUIT,NETO 10.5,IVA 10.5,NETO 21,IVA 21,TOTAL_NO_GRAVADO
0,01/09/2025,00080-00280200,ASSIST CARD ARGENTINA SOCIEDAD ANON,33547992429,0.0,0.0,0.0,0.0,700114.40
1,01/09/2025,00008-00083337,CARNIVAL SRL,30676579938,263215.24,27637.6,0.0,0.0,14317.78
2,01/09/2025,00008-00083129,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,4116040.00
3,01/09/2025,00008-00083148,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,1776160.00
4,01/09/2025,00008-00083149,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,123896.00
...,...,...,...,...,...,...,...,...,...
139,13/09/2025,00008-00085781,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,344777.00
140,13/09/2025,00008-00085787,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,299100.55
141,13/09/2025,00080-00281832,ASSIST CARD ARGENTINA SOCIEDAD ANON,33547992429,0.0,0.0,0.0,0.0,292699.20
142,13/09/2025,00008-00085785,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,187965.60
